In [1]:
from sklearn.datasets import load_breast_cancer
import pandas as pd
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr
from rpy2.robjects.conversion import localconverter
import numpy as np
from sklearn.linear_model import LogisticRegression
from statsmodels.discrete.discrete_model import Logit
from statsmodels.tools.tools import add_constant
from sklearn.linear_model import SGDClassifier
import random
from sklearn.metrics import accuracy_score
import statsmodels.api as sm
from sklearn.metrics import log_loss
from statsmodels.tools.tools import add_constant
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from statsmodels.genmod.generalized_linear_model import GLM
from utils import _add_constant,_hat_diag,_sigmoid_pred,_sigmoid_pred, _information_matrix, _predict, _predict_proba
#import R package brglm
base = importr('base')
d = {'package.dependencies': 'package_dot_dependencies',
     'package_dependencies': 'package_uscore_dependencies'}
brglm = importr('brglm',robject_translations=d)

/Users/remycanario/anaconda3/lib/python3.7/site-packages/statsmodels/compat/pandas.py:23: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


In [2]:
#Load Data
features = [i.replace(' ', '_') for i in load_breast_cancer().feature_names.tolist()]

breast_cancer_df = pd.DataFrame(load_breast_cancer().data,columns=features)
target_df = pd.DataFrame(load_breast_cancer().target, columns=['y'])
X = breast_cancer_df.iloc[:,:5]
y = target_df

df = pd.concat([target_df,breast_cancer_df],axis=1).iloc[:,:6]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2,random_state=0)

In [3]:
# Create small data set and rare event data set
num_successes_for_rare = int(((((1-df.y.mean())*df.shape[0])/.95)-((1-df.y.mean())*df.shape[0]))//1)
rare_inds = sorted(list(df[df.y==0].index) + random.sample(list(df[df.y==1].index),num_successes_for_rare))
# small_inds = random.sample(sorted(list(df.index)),50)


rare_df = df.iloc[rare_inds,:6]
rare_X = rare_df.drop('y',axis=1)
rare_y = rare_df['y']

separation_df = pd.read_csv('separation_df.csv',index_col=0).iloc[:,]
separation_X = separation_df.drop('y',axis=1)
separation_y = separation_df['y']

small_df = pd.read_csv('small_df.csv',index_col=0)
small_X = small_df.drop('y',axis=1)
small_y = small_df['y']


In [1311]:
#Helper functions
def formula_from_df(df,y_var_name):
    features = list((df.drop(y_var_name,axis=1).columns))
    formula = y_var_name + '~' + ' + '.join(features)
    return formula

def Sigmoid_Pred(X, weights):
    z = np.dot(X,weights)
    sig =  1/(1 + np.exp(-1*z))
    sig = np.clip(sig,.000001,.999999)
    return sig

def hat_diag(X,weights):
    Xt = X.transpose()
    
    #Get diagonal of error
    y_pred = Sigmoid_Pred(X,weights)
    W = np.diag(y_pred*(1-y_pred))
   
    #Calculate Fisher Information Matrix
    I = np.linalg.multi_dot([Xt,W,X]) 

    #Get Diagonal of Hat Matrix
    hat = np.linalg.multi_dot([W**0.5,X,np.linalg.inv(I),Xt,W**0.5])
    hat_diag = np.diag(hat)
    return hat_diag

def marginal_effects(X,weights):
    #at means
    column_means = X.mean()
    p = Sigmoid_Pred(column_means,weights)
    at_means = np.ones(6)
    for i in range(weights.shape[0]):
        weights_copy = c.copy()
        weights_copy[i]+=1
        new_p =Sigmoid_Pred(means,weights_copy)
        at_means[i] = new_p-p
    
    #meaned
    averaged_marg_effs = np.ones((X.shape[0],X.shape[1]))
    for i in range(X.shape[0]):
        row = X.iloc[i]
        p = Sigmoid_Pred(row,c)
        for j in range(weights.shape[0]):
            weights_copy = weights.copy()
            weights_copy[j]+=1
            new_p = 1/(1+np.exp(-np.dot(row,weights_copy)))
            eff = new_p-p
            averaged_marg_effs[i,j] = eff
        ame = pd.DataFrame(averaged_marg_effs.mean(axis=0),index=X.columns, columns=['mean'])
        ame['at_means'] = at_means
    return ame

def information_matrix(X,weights):
    Xt = X.transpose()

    #Get diagonal of error
    y_pred = Sigmoid_Pred(X,weights)
    W = np.diag(y_pred*(1-y_pred))

    #Calculate Fisher Information Matrix
    I = np.linalg.multi_dot([Xt,W,X])
    return I

In [ ]:
y_pred = self.Sigmoid_Pred(X, weights)
W = np.diag(y_pred*(1-y_pred))
Xt = X.transpose()
I = np.linalg.multi_dot([Xt,W,X]) 
hat = np.linalg.multi_dot([W**0.5,X,np.linalg.inv(I),Xt,W**0.5])
hat_diag = np.diag(hat)

In [1335]:
#firth from scratch
def firth_logit(X,y,num_iter=10000,alpha=0.01,add_int=True):
    #add intercept if necessary
    if add_int==True:
        X = add_constant(X)
    
    #initialize weights
    weights=np.ones(X.shape[1])
    
    #Perform gradient descent
    for i in range(num_iter):
        y_pred = Sigmoid_Pred(X,weights)
        H = hat_diag(X,weights)
        #Update weights
        U = np.matmul((y -y_pred + H*(0.5 - y_pred)),X)
        weights += np.matmul(np.linalg.inv(I),U)*alpha
        if (i%1000==0):
            print('log likelihood: ',(y*np.log(y_pred)+(1-y)*np.log(1-y_pred)).sum()+0.5*np.log(np.linalg.det(I)))
    preds = Sigmoid_Pred(X,weights)
    weights = pd.Series(weights,index=X.columns)
    return preds, weights

In [1276]:
def firth_logit_r(df,y_var_name, formula, r_data=True, weights='none',all_coef_summary=False):
    #convert data frame to R df
    if r_data==False:
        if type(weights) ==str:
            with localconverter(ro.default_converter + pandas2ri.converter):
                df = ro.conversion.py2ri(df)
        else:
            with localconverter(ro.default_converter + pandas2ri.converter):
                df = ro.conversion.py2ri(df)
                weights = ro.vectors.FloatVector(weights)
    
    #create firth logit model
    if weights!='none':
        model = brglm.brglm(formula, data = df, family='binomial',pl=True, weights=weights)
    else:
        model = brglm.brglm(formula, data = df, family='binomial',pl=True)
    
    #extract coefficients
    summary = base.summary(model)
    summary_dic = {}
    for i in range(len(summary.names)):
        try:
            summary_dic[summary.names[i]]=pandas2ri.converter.ri2py(list(summary)[i])
        except:
            pass
    columns = list(df.colnames)
    columns[0]='Intercept'
    if all_coef_summary==True:
        coefs = pd.DataFrame(summary_dic['coefficients'],columns=(['Coef','SE','Z','P']),index=columns)
    else:
        coefs = pd.DataFrame(summary_dic['coefficients'],columns=(['Coef','SE','Z','P']),index=columns).Coef
    
    #get raw output and apply sigmoid
    preds = ro.r.predict(model,df)
    preds = 1/(1+np.exp(-np.array(preds)))       
    return preds, coefs

In [1300]:
#firth with hyperparameter
def tuned_firth_logit(X,y,lmbda=0.5,num_iter=10000,alpha=0.01,add_int=True):
    #add intercept if necessary
    if add_int==True:
        X = add_constant(X)
    
    #initialize weights
    weights=np.ones(X.shape[1])
    
    #Perform gradient descent
    for i in range(num_iter):
        y_pred = Sigmoid_Pred(X,weights)
        H = hat_diag(X,weights)
        U = np.matmul((y -y_pred + lmbda*H*(1 - 2*y_pred)),X)
        weights += np.matmul(np.linalg.inv(I),U)*alpha
        if (i%1000==0):
            print('log likelihood: ',(y*np.log(y_pred)+(1-y)*np.log(1-y_pred)).sum()+lmbda*np.log(np.linalg.det(I)))
    return pd.Series(weights,index=X.columns)

In [1224]:
def FLIC(X,y,num_iter=10000,lmbda=0.5):
    #get firth logit coefs
    coefs = tuned_firth_logit(X,y,lmbda=lmbda,num_iter=num_iter)
    
    #reestimate intercept
    coefs.drop('const',inplace=True)
    eta = np.dot(X,coefs)
    target = y-eta
    b0_model = sm.OLS(target,np.ones(y.shape[0])).fit()
    b0 = b0_model.params[0]
    coefs = pd.Series(b0,index=['Int']).append(coefs)
    
    #get predictions
    X=add_constant(X)
    preds = Sigmoid_Pred(X.values,coefs.values)
    return preds,coefs

In [1223]:
def FLIC_brglm(df, y_var_name):
    formula = formula_from_df(df,y_var_name)
    with localconverter(ro.default_converter + pandas2ri.converter):
        df_r = ro.conversion.py2ri(df)
    model = brglm.brglm(formula, data = df_r, family='binomial',pl=True)
    summary = base.summary(model)
    summary_dic = {}
    for i in range(len(summary.names)):
        try:
            summary_dic[summary.names[i]]=pandas2ri.converter.ri2py(list(summary)[i])
        except:
            pass
    columns = list(firth_small_r.colnames)
    coefs = pd.DataFrame(summary_dic['coefficients'],columns=(['Coef','SE','Z','P']),index=columns).Coef[1:]
    y_var_name = formula.split('~')[0].strip()
    y = df[y_var_name].values
    X = df.drop(y_var_name,axis=1)
    eta = np.dot(X,coefs)
    target = y-eta
    b0_model = sm.OLS(target,np.ones(y.shape[0])).fit()
    b0 = pd.Series(b0_model.params[0],index=['Int'])
    coefs = b0.append(coefs)
    X=add_constant(X)
    preds = Sigmoid_Pred(X.values,coefs.values)
    return preds,coefs

In [1287]:
def FLAC_brglm(df,y_var_name):
    
    init_rows = df.shape[0]
    X = add_constant(df.drop(y_var_name,axis=1))
    y = df[y_var_name]
    
    #Build Hat Matrix = (W**0.5)*X*((XtWX)^-1)*Xt*W**0.5
    model = sm.Logit(y,X).fit()
    weights = model.params
    H = hat_diag(X,weights)
    
    #Duplicate every row
    double_df = df.append(df)
    
    #Create a new copy of the original data
    pseudo_y_df = df
    #Change y to 1-y
    pseudo_y_df[y_var_name]=1-pseudo_y_df[y_var_name]
    
    #Append to doubled df
    aug_df = double_df.append(pseudo_y_df)
    
    #Create dummy for real vs. duplicated/pseudo data
    aug_df['real_data'] = 0
    aug_df['real_data'][init_rows:]=1
    
    
    #Create regression formula
    formula = formula_from_df(aug_df,y_var_name)
    
    
    #Create vector of weights = 1 for real data, hi/2 for augmentation data
    aug_sample_weights = pd.Series(np.concatenate([np.ones(init_rows),H/2,H/2]))
    
    
    #Get predictions and coefficients
    preds, coefs = firth_logit_r(df=aug_df,
                                 y_var_name=y_var_name,
                                 formula=formula,
                                 r_data=False,
                                 weights=aug_sample_weights)
    
    return preds, coefs
    

In [1282]:
def logF11(df,y_var_name,intercept=False):
    '''Perform log-f(1,1) data augmentation
       Returns augmented df and observation weights'''
    
    num_rows = 2*(df.shape[1]-1)
    y_ind = df.columns.get_loc(y_var_name)
    
    aug = pd.DataFrame(0,columns=df.columns,index=(range(num_rows)))
    
    #augment y variable
    aug.iloc[range(0,num_rows,2),y_ind]=1
    y = aug[y_var_name]
    
    #augment X variables
    X = aug.drop(y_var_name,axis=1)
    for ind, rows in enumerate(range(0,X.shape[0],2)):
         X.iloc[rows:rows+2,ind]=1
    
    #bring it all together
    aug = pd.concat([y,X],axis=1)
    f_df = df.append(aug)
    
    #add offset
    f_df['real_data']=1
    f_df['real_data'][-aug.shape[0]:]=0
    
    #reseparate
    X = f_df.drop(y_var_name,axis=1)
    y = f_df[y_var_name]
    
    #Calculate weights
    weights = f_df['real_data'].apply(lambda x: 0.5 if x == 0 else 1)
    model = sm.Logit(y,X).fit()
    coefs = model.params
    if intercept==True:
        eta = np.dot(X,coefs)
        target = y-eta
        b0_model = sm.OLS(target,np.ones(y.shape[0])).fit()
        b0 = pd.Series(b0_model.params[0],index=['Int'])
        coefs = b0.append(coefs)
        X=add_constant(X)

    preds = Sigmoid_Pred(X.values,coefs)
    return preds, coefs

Intermediate versions that may be worth keeping 

In [790]:
#log-F(1,1) just augmentation
def logF11_aug(df,y_var_name,R=False):
    '''Perform log-f(1,1) data augmentation
       Returns augmented df and observation weights'''
    
    num_rows = 2*(df.shape[1]-1)
    y_ind = df.columns.get_loc(y_var_name)
    
    aug = pd.DataFrame(0,columns=df.columns,index=(range(num_rows)))
    
    #augment y variable
    aug.iloc[range(0,num_rows,2),y_ind]=1
    y = aug[y_var_name]
    
    #augment X variables
    X = aug.drop(y_var_name,axis=1)
    for ind, rows in enumerate(range(0,X.shape[0],2)):
         X.iloc[rows:rows+2,ind]=1
    
    #bring it all together
    aug = pd.concat([y,X],axis=1)
    f_df = df.append(aug)
    
    #add offset
    f_df['real_data']=1
    f_df['real_data'][-aug.shape[0]:]=0
    
    #Calculate weights
    weights = f_df['real_data'].apply(lambda x: 0.5 if x == 0 else 1)
    if R==True:
        with localconverter(ro.default_converter + pandas2ri.converter):
    
            f_df = ro.conversion.py2ri(f_df)
            weights = ro.vectors.FloatVector(weights)
    return f_df, weights      

In [787]:
#FLAC just augmentation
def FLAC_aug(df,y_var_name,R=False):
    '''Perform FLAC data augmentation
       Returns augmented df and observation weights'''
    
    init_rows = df.shape[0]
    X = add_constant(df.drop(y_var_name,axis=1))
    y = df[y_var_name]
    glm = GLM(y,X,family=sm.families.Binomial()).fit()
#     hat = glm.get_hat_matrix_diag()
    weights = glm.params
    y_pred = glm.predict(test_X)
    W = np.diag(y_pred*(1-y_pred))
    test_XtWtest_X = np.linalg.multi_dot([test_X.transpose(),W,test_X])
    I = np.linalg.inv(test_XtWtest_X)
    hat = np.diag(np.linalg.multi_dot([W**0.5,test_X,I,test_X.transpose(),W**0.5]))

    aug_df = pd.concat([df,df,df])
    aug_df[y_var_name][init_rows*2:]=1-aug_df[y_var_name][init_rows*2:]
    aug_df['pseudo_data'] = 0
    aug_df['pseudo_data'][init_rows:]=1

    aug_sample_weights = pd.Series(np.concatenate([np.ones(init_rows),hat/2,hat/2]))
    if R==True:
        with localconverter(ro.default_converter + pandas2ri.converter):
    
            aug_df = ro.conversion.py2ri(aug_df)
            aug_sample_weights = ro.vectors.FloatVector(aug_sample_weights)
    return aug_df, aug_sample_weights
    
    #Now run this through brglm

In [785]:
#created augmented datasets

#log-f(1,1)
# all_f, all_weights_f = logf11_aug(all_df,'y')
small_f, small_weights_f = logf11_aug(small_df,'y')
rare_f, rare_weights_f = logf11_aug(rare_df,'y')
separation_f, separation_weights_f = logf11_aug(separation_df,'y')

#FLAC
# all_FLAC, all_weights_FLAC = FLAC_aug(all_df,'y')
small_FLAC, small_weights_FLAC = FLAC_aug(small_df,'y')
rare_FLAC, rare_weights_FLAC = FLAC_aug(rare_df,'y')
separation_FLAC, separation_weights_FLAC = FLAC_aug(separation_df,'y')

#created R augmented datasets

#log-f(1,1)
# all_f, all_weights_f = logf11_aug(all_df,'y')
small_f_r, small_weights_f_r = logf11_aug(small_df,'y')
rare_f_r, rare_weights_f_r = logf11_aug(rare_df,'y')
separation_f_r, separation_weights_f_r = logf11_aug(separation_df,'y')

#FLAC
# all_FLAC, all_weights_FLAC = FLAC_aug(all_df,'y')
small_FLAC_r, small_weights_FLAC_r = FLAC_aug(small_df,'y')
rare_FLAC_r, rare_weights_FLAC_r = FLAC_aug(rare_df,'y')
separation_FLAC_r, separation_weights_FLAC_r = FLAC_aug(separation_df,'y')

/Users/remycanario/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/remycanario/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/remycanario/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/remycanario/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set o

WIP

In [ ]:
def boosted_firth(X,y,num_steps=500):
    init_rows = X.shape[0]
    aug_X = X.append(X)
    aug_y = y.append(1-y)
    eta0=np.full((1,,sm.Logit(aug_y,aug_X.const).fit().params[0])
    weights = np.ones(aug_X.shape[1])
    for i in range(num_steps):
        hat_diag = hat(aug_X,weights)
        y_pred = Sigmoid_Pred(aug_X,weights)
        offset = (1+h/2)*(aug_y-y_pred*())

In [1325]:
def testing(df,y_var_name,X_train,y_train,X_test,y_test):
    models = ['logit','l2','firth','FLIC','FLAC','log-f(1,1)','log-f(1,1)_with_int']

    c_X_train = add_constant(X_train)
    c_X_test = add_constant(X_test)
    
    base = sm.Logit(y_train,c_X_train).fit()
    control_proba = base.predict(c_X_test)
    control_preds = control_proba.round()
    
    l2 = LogisticRegression()
    l2.fit(X_train,y_train)
    l2_proba = l2.predict_proba(X_test)
    l2_preds = l2.predict(X_test)
    
    firth_proba, firth_coefs = firth_logit(X_train,y_train)
    firth_preds = firth_proba.round()
    
    FLIC_proba, FLIC_coefs = FLIC(X_train,y_train)
    FLIC_preds = FLIC_proba.round()
    
    FLAC_proba, FLAC_coefs = FLAC_brglm(df,y_var_name)
    FLAC_preds = FLAC_proba.round()
    
    logF11_proba, logF11_coefs = logF11(df,y_var_name)
    logF11_preds = logF11_proba.round()
    
    logF11_int_proba, logF11_coefs = logF11(df,y_var_name,intercept=True)
    logF11_int_preds = logF11_int_round()
    
    proba = [control_proba, l2_proba, firth_proba, FLIC_proba, FLAC_proba, logF11_proba, logF11_int_proba]
    preds = [control_preds, l2_preds, firth_preds, FLIC_preds, FLAC_preds, logF11_preds, logF11_int_preds]
    return proba, preds
    
    

In [111]:
#Firth Class
class PMLE():
    class Firth_Logit():
        def __init__(self,num_iters=10000, alpha=0.01,add_int=True,lmbda=0.5,FLIC=False):

            self.alpha = alpha
            self.num_iters = num_iters
            self.add_int = add_int
            self.lmbda=lmbda
            self.FLIC=FLIC
        
        def firth_gd(self,X,y,weights):
            y_pred = _sigmoid_pred(X=X,weights=weights)
            H =_hat_diag(X,weights)
            I = _information_matrix(X,weights)
            U = np.matmul((y -y_pred + self.lmbda*H*(1 - 2*y_pred)),X)
            weights += np.matmul(np.linalg.inv(I),U)*self.alpha
            return weights
        
        def fit(self,X,y):
            #add intercept if necessary
            orig_X = X
            if self.add_int==True:
                X =_add_constant(X)
            self.X = X
            self.y = y

            #initialize weights
            weights=np.ones(X.shape[1])

            #Perform gradient descent
            for i in range(self.num_iters):
                weights = self.firth_gd(X,y,weights)
            
            if self.FLIC==True:
                weights = weights[1:]
                eta = np.dot(orig_X,weights)
                target = y-eta
                b0_model = sm.OLS(target,np.ones(y.shape[0])).fit()
                b0 = b0_model.params[0]
                weights = np.insert(weights,0,b0)
            
            weights = pd.Series(weights,index=X.columns)
            self.weights = weights
            
            self.I = _information_matrix(X,weights)
            self.hat_matrix_diag = _hat_diag(X,weights)
            self.Hessian = -self.I
            
            
        def marginal_effects(self):
            #at means
            column_means = self.X.mean()
            p = _sigmoid_pred(column_means,self.weights)
            at_means = np.ones(6)
            for i in range(self.weights.shape[0]):
                self.weights_copy = self.weights.copy()
                self.weights_copy[i]+=1
                new_p =_sigmoid_pred(column_means,self.weights_copy)
                at_means[i] = new_p-p

            #meaned
            averaged_marg_effs = np.ones((self.X.shape[0],self.X.shape[1]))
            for i in range(self.X.shape[0]):
                row = self.X.iloc[i]
                p = _sigmoid_pred(row,self.weights)
                for j in range(self.weights.shape[0]):
                    self.weights_copy = self.weights.copy()
                    self.weights_copy[j]+=1
                    new_p = 1/(1+np.exp(-np.dot(row,self.weights_copy)))
                    eff = new_p-p
                    averaged_marg_effs[i,j] = eff
                ame = pd.DataFrame(averaged_marg_effs.mean(axis=0),index=self.X.columns, columns=['mean'])
                ame['at_means'] = at_means
            return ame
        
        def predict_proba(self,X):
            if X.shape[1]==self.X.shape[1]-1:
                X = _add_constant(X)
            preds = _sigmoid_pred(X,self.weights)
            return preds

        def predict(self,X):
            if X.shape[1]==self.X.shape[1]-1:
                X = _add_constant(X)
            preds = _sigmoid_pred(X,self.weights).round()
            return preds
    
    class logF11():
        def __init__(self,intercept=False):
            self.intercept=False
        
        def data_augementation(self,df,y_var_name):
            num_rows = 2*(df.shape[1]-1)
            y_ind = df.columns.get_loc(y_var_name)

            aug = pd.DataFrame(0,columns=df.columns,index=(range(num_rows)))

            #augment y variable
            aug.iloc[range(0,num_rows,2),y_ind]=1
            y = aug[y_var_name]

            #augment X variables
            X = aug.drop(y_var_name,axis=1)
            for ind, rows in enumerate(range(0,X.shape[0],2)):
                 X.iloc[rows:rows+2,ind]=1

            #bring it all together
            aug = pd.concat([y,X],axis=1)
            f_df = df.append(aug)

            #add offset
            f_df['real_data']=1
            f_df['real_data'][-aug.shape[0]:]=0
            f_df['real_data'].apply(lambda x: 0.5 if x == 0 else 1)

            #reseparate
            X = f_df.drop(y_var_name,axis=1)
            y = f_df[y_var_name]
            
            self.X = X
            self.y = y
    
            return X, y
        
        def fit(self,df,y_var_name):
            X, y = self.data_augementation(df,y_var_name)
            model = sm.Logit(y,X).fit()
            weights = model.params
            if self.intercept==True:
                eta = np.dot(X,weights)
                target = y-eta
                b0_model = sm.OLS(target,np.ones(y.shape[0])).fit()
                b0 = b0_model.params[0]
                weights = np.insert(weights,0,b0)
                X = _add_constant(X)
            self.X = X
            weights = pd.Series(weights,index=X.columns)
            self.weights = weights
        
        def predict(self,X):
            return _predict(X,self.weights)
        
        def predict_proba(self,X):
            return _predict_proba(X,self.weights)
            



In [ ]:
if i==num_iter:
    print('log likelihood: ',(y*np.log(y_pred)+(1-y)*np.log(1-y_pred)).sum()+0.5*np.log(np.linalg.det(I)))

In [ ]:
#figure out sample weights
def _logistic_loss_and_grad(w, X, y, alpha, sample_weight=None):
    """Computes the logistic loss and gradient.
    Parameters
    ----------
    w : ndarray of shape (n_features,) or (n_features + 1,)
        Coefficient vector.
    X : {array-like, sparse matrix} of shape (n_samples, n_features)
        Training data.
    y : ndarray of shape (n_samples,)
        Array of labels.
    alpha : float
        Regularization parameter. alpha is equal to 1 / C.
    sample_weight : array-like of shape (n_samples,), default=None
        Array of weights that are assigned to individual samples.
        If not provided, then each sample is given unit weight.
    Returns
    -------
    out : float
        Logistic loss.
    grad : ndarray of shape (n_features,) or (n_features + 1,)
        Logistic gradient.
    """
    n_samples, n_features = X.shape
    grad = np.empty_like(w)

    w, c, yz = _intercept_dot(w, X, y)

    if sample_weight is None:
        sample_weight = np.ones(n_samples)

    # Logistic loss is the negative of the log of the logistic function.
    out = -np.sum(sample_weight * log_logistic(yz)) + .5 * alpha * np.dot(w, w)

    z = expit(yz)
    z0 = sample_weight * (z - 1) * y

    grad[:n_features] = safe_sparse_dot(X.T, z0) + alpha * w

    # Case where we fit the intercept.
    if grad.shape[0] > n_features:
        grad[-1] = z0.sum()
    return out, grad

def safe_sparse_dot(a, b, dense_output=False):
    """Dot product that handle the sparse matrix case correctly
    Parameters
    ----------
    a : array or sparse matrix
    b : array or sparse matrix
    dense_output : boolean, (default=False)
        When False, ``a`` and ``b`` both being sparse will yield sparse output.
        When True, output will always be a dense array.
    Returns
    -------
    dot_product : array or sparse matrix
        sparse if ``a`` and ``b`` are sparse and ``dense_output=False``.
    """
    if a.ndim > 2 or b.ndim > 2:
        if sparse.issparse(a):
            # sparse is always 2D. Implies b is 3D+
            # [i, j] @ [k, ..., l, m, n] -> [i, k, ..., l, n]
            b_ = np.rollaxis(b, -2)
            b_2d = b_.reshape((b.shape[-2], -1))
            ret = a @ b_2d
            ret = ret.reshape(a.shape[0], *b_.shape[1:])
        elif sparse.issparse(b):
            # sparse is always 2D. Implies a is 3D+
            # [k, ..., l, m] @ [i, j] -> [k, ..., l, j]
            a_2d = a.reshape(-1, a.shape[-1])
            ret = a_2d @ b
            ret = ret.reshape(*a.shape[:-1], b.shape[1])
        else:
            ret = np.dot(a, b)
    else:
        ret = a @ b

    if (sparse.issparse(a) and sparse.issparse(b)
            and dense_output and hasattr(ret, "toarray")):
        return ret.toarray()
    return ret

def _intercept_dot(w, X, y):
    """Computes y * np.dot(X, w).
    It takes into consideration if the intercept should be fit or not.
    Parameters
    ----------
    w : ndarray of shape (n_features,) or (n_features + 1,)
        Coefficient vector.
    X : {array-like, sparse matrix} of shape (n_samples, n_features)
        Training data.
    y : ndarray of shape (n_samples,)
        Array of labels.
    Returns
    -------
    w : ndarray of shape (n_features,)
        Coefficient vector without the intercept weight (w[-1]) if the
        intercept should be fit. Unchanged otherwise.
    c : float
        The intercept.
    yz : float
        y * np.dot(X, w).
    """
    c = 0.
    if w.size == X.shape[1] + 1:
        c = w[-1]
        w = w[:-1]

    z = safe_sparse_dot(X, w) + c
    yz = y * z
    return w, c, yz

def log_logistic(X, out=None):
    """Compute the log of the logistic function, ``log(1 / (1 + e ** -x))``.
    This implementation is numerically stable because it splits positive and
    negative values::
        -log(1 + exp(-x_i))     if x_i > 0
        x_i - log(1 + exp(x_i)) if x_i <= 0
    For the ordinary logistic function, use ``scipy.special.expit``.
    Parameters
    ----------
    X : array-like, shape (M, N) or (M, )
        Argument to the logistic function
    out : array-like, shape: (M, N) or (M, ), optional:
        Preallocated output array.
    Returns
    -------
    out : array, shape (M, N) or (M, )
        Log of the logistic function evaluated at every point in x
    Notes
    -----
    See the blog post describing this implementation:
    http://fa.bianp.net/blog/2013/numerical-optimizers-for-logistic-regression/
    """
    is_1d = X.ndim == 1
    X = np.atleast_2d(X)
    X = check_array(X, dtype=np.float64)

    n_samples, n_features = X.shape

    if out is None:
        out = np.empty_like(X)

    _log_logistic_sigmoid(n_samples, n_features, X, out)

    if is_1d:
        return np.squeeze(out)
    return out

In [1586]:
pd.Series(I.mean(axis=0))

0    3.191122e+03
1    4.522077e+04
2    6.358897e+04
3    2.941959e+05
4    1.999059e+06
5    3.126354e+02
dtype: float64

In [1587]:
U

const                2.749730
mean_radius         28.778599
mean_texture        52.892120
mean_perimeter     187.054709
mean_area          880.750305
mean_smoothness      0.299444
Name: y, dtype: float64